In [4]:
#3 Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [5]:
#3 Define path

path = r'/Users/shaheirjibin/Documents/Data Analytics/02 Data Analytics Immersion/04 Python Fundamentals for Data Analysts/Instacart Basket Analysis'

In [6]:
#3 Import 'customers' dataframe

customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [7]:
# Preview the imported df

customers.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [8]:
customers.shape

(206209, 10)

In [9]:
customers.describe()

user_id            Age   n_dependants         income
count  206209.000000  206209.000000  206209.000000  206209.000000
mean   103105.000000      49.501646       1.499823   94632.852548
std     59527.555167      18.480962       1.118433   42473.786988
min         1.000000      18.000000       0.000000   25903.000000
25%     51553.000000      33.000000       0.000000   59874.000000
50%    103105.000000      49.000000       1.000000   93547.000000
75%    154657.000000      66.000000       3.000000  124244.000000
max    206209.000000      81.000000       3.000000  593901.000000

In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [25]:
#4 Rename the variables so that they look consistent

customers.rename(columns = {'First Name' : 'first_name', 
                            'Surnam' : 'last_name',
                            'Gender' : 'gender',
                            'STATE' : 'state', 
                            'Age' : 'age', 
                            'n_dependants' : 'dependants', 
                            'family_status' : 'marital_status'}, inplace = True)

In [28]:
#4 Check newly renamed variables

customers.head()

user_id first_name last_name  gender       state  age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   dependants marital_status  income  
0           3        married  165665  
1           0         single   59285  
2           2        married   99568  
3           0         single   42049  
4           1        married   40374

7 columns have been renamed for consistency. No columns were dropped as of now.

In [29]:
#5 Check for missing values

customers.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
dependants            0
marital_status        0
income                0
dtype: int64

The 11259 missing first names are not crucial for this analysis, so they were left as is.

In [30]:
#5 Check for duplicates

df_dups = customers[customers.duplicated()]

In [31]:
df_dups

Empty DataFrame
Columns: [user_id, first_name, last_name, gender, state, age, date_joined, dependants, marital_status, income]
Index: []

No duplicates were found in the 'customers' dataframe.

In [32]:
#5 Check for mixed-type

customers.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
dependants         int64
marital_status    object
income             int64
dtype: object

In [33]:
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


The variable 'first_name' has mixed data type.

In [34]:
#5 Address all 'first_name' data type to string

customers['first_name'] = customers['first_name'].astype('str')

In [35]:
#5 Change the variable 'date_joined' from string to datetime

customers['date_joined']=pd.to_datetime(customers['date_joined'])

In [37]:
#5 View data types again

customers.dtypes

user_id                    int64
first_name                object
last_name                 object
gender                    object
state                     object
age                        int64
date_joined       datetime64[ns]
dependants                 int64
marital_status            object
income                     int64
dtype: object

# 6. Combining the customer data with the rest of the prepared Instacart data

In [44]:
#6 Import orders_products_flags data

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_flags.pkl'))

In [45]:
#6 Check data type

ords_prods_merge.dtypes

order_id                            int64
user_id                             int64
order_number                        int64
orders_day_of_week                  int64
order_hour_of_day                   int64
days_since_prior_order            float64
product_id                          int64
add_to_cart_order                   int64
reordered                           int64
product_name                       object
aisle_id                            int64
department_id                       int64
prices                            float64
_merge                           category
price_range_loc                    object
busiest_day                        object
busiest_days                       object
busiest_period_of_day              object
max_order                           int64
loyalty_flag                       object
mean_product_price                float64
spending_flag                      object
median_days_since_prior_order     float64
order_frequency_flag              

In [56]:
#6 Drop '_merge' variable

ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])

In [50]:
#6 Change 'user_id' in both dataframes data type from integer to string

ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')
customers['user_id'] = customers['user_id'].astype('str')

The key variable is 'user_id'.

In [51]:
#6 Check data type for 'user_id' in 'customers' dataframe

customers.dtypes['user_id']

dtype('O')

In [52]:
#6 Check data type for 'user_id' in 'ords_prods_merge' dataframe

ords_prods_merge.dtypes['user_id']

dtype('O')

In [57]:
#6 Merge the two dataframes

ords_prods_cust_merge = ords_prods_merge.merge(customers, on = 'user_id', indicator = True)

In [58]:
# View merged dataframe

ords_prods_cust_merge.head()

order_id user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329       1             1                   2                  8   
1   2539329       1             1                   2                  8   
2   2539329       1             1                   2                  8   
3   2539329       1             1                   2                  8   
4   2539329       1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     0.0         196                  1          0   
1                     0.0       14084                  2          0   
2                     0.0       12427                  3          0   
3                     0.0       26088                  4          0   
4                     0.0       26405                  5          0   

                              product_name  ...  first_name  last_name  \
0                                     Soda  ...       Linda     Nguyen   
1  Organic Unsweetened Vanilla Almond Milk  ...       Linda     Nguyen   
2                      Original Beef Jerky  ...       Linda     Nguyen   
3               Aged White Cheddar Popcorn  ...       Linda     Nguyen   
4         XL Pick-A-Size Paper Towel Rolls  ...       Linda     Nguyen   

   gender    state age date_joined dependants  marital_status income  _merge  
0  Female  Alabama  31  2019-02-17          3         married  40423    both  
1  Female  Alabama  31  2019-02-17          3         married  40423    both  
2  Female  Alabama  31  2019-02-17          3         married  40423    both  
3  Female  Alabama  31  2019-02-17          3         married  40423    both  
4  Female  Alabama  31  2019-02-17          3         married  40423    both  

[5 rows x 33 columns]

In [59]:
#8 Export merged dataframe as pickle

ords_prods_cust_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))